# Обучение нейросетевых моделей

В этом файле мы обучаем модели глубокого обучения для 2-х NER (Named Entity Recognition) задач:

 - Разбивку имени на составляющие (Фамилия, имя, отчество) - используем в качестве основы модель Bert-base
 - Разбивку адреса на составляющие (регион, район, город, ...) - используем в качестве основы модель rubert-base-cased

Обе модели тонко настраиваются на специаально сгенерированных наборах данных. Генераторы наборов данных описаны в jupyter-ноутбуке Data Parser.

А также загружаем модель M2M100 для исправления ошибок. Так как полноценное обучение и даже тонкая настройка данной модели представляет некоторую сложность, было принято решение провести квантизацию данной модели.

## Модули

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import time
from tqdm.auto import tqdm

import datetime
import os
import re

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoModelForTokenClassification, AutoTokenizer, \
    pipeline, DataCollatorForTokenClassification, get_scheduler

from accelerate import Accelerator

import torch
from torch.optim import AdamW
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

import evaluate

## Исправление орфографии

### Квантизация модели

Так как базовая модель занимает достаточно много места (около 4.3 ГБ), мы можем провести квантизацию модели, чтобы уменьшить её размер на диске.  

В целом, квантизация модели не должна приводить к значимой потери качества её предсказаний.

In [12]:
# path_to_model = "ai-forever/RuM2M100-1.2B" 

path_to_model = "model/M2M100ForConditionalGeneration/" 
path_to_tokenizer = "model/M2M100Tokenizer/"

In [13]:
model_M100_spell = M2M100ForConditionalGeneration.from_pretrained(path_to_model)
tokenizer_M100_spell = M2M100Tokenizer.from_pretrained(path_to_tokenizer)

In [ ]:
# model_M100_spell.save_pretrained("model/M2M100ForConditionalGeneration/")
# tokenizer_M100_spell.save_pretrained("model/M2M100Tokenizer/")

In [14]:
def calc_size(model):

    '''
    Calculates size of the model
    '''
    
    torch.save(model.state_dict(), "./tmp/model.p")
    size=os.path.getsize("./tmp/model.p")
    os.remove('./tmp/model.p')
    return "{:.3f} KB".format(size / 1024)

In [15]:
calc_size(model_M100_spell)

'4386893.908 KB'

In [16]:
quantized_model = torch.quantization.quantize_dynamic(model_M100_spell, {torch.nn.Linear}, dtype=torch.qint8)

In [17]:
calc_size(quantized_model)

'1157517.419 KB'

Модель достаточно сильно сжалась - примерно в 4 раза!

In [19]:
torch.save(quantized_model, "model/M2M100_spellchecker/quantized_model.pt")
tokenizer_M100_spell.save_pretrained("model/M2M100_tokenizer/")

('model/M2M100_tokenizer/tokenizer_config.json',
 'model/M2M100_tokenizer/special_tokens_map.json',
 'model\\M2M100_tokenizer\\vocab.json',
 'model\\M2M100_tokenizer\\sentencepiece.bpe.model',
 'model/M2M100_tokenizer/added_tokens.json')

### Тест

Проверим, что квантизованная модель приемлемо работает на наборе данных из формы и нескольких произвольных примерах.

In [3]:
model_M100_spell = torch.load("model/M2M100_spellchecker/quantized_model.pt")
model_M100_spell.eval()

c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:314: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(14341, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(14341, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): DynamicQuantizedLinear(in_features=1024, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v_proj): DynamicQuantizedLinear(in_features=1024, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (q_proj): DynamicQuantizedLinear(in_features=1024, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (out_proj): DynamicQuantizedLinear(in_features=1024, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affi

In [4]:
path_to_tokenizer = "model/M2M100_tokenizer/"
tokenizer_M100_spell = M2M100Tokenizer.from_pretrained(path_to_tokenizer)

In [5]:
def correct_errors(sentence_in: str) -> str:

    """
    Функция для исправления орфографии в модели 

    Параметры:
    sentence_in : str
        Предложение с (возможно) орфографическими ошибками

    Возвращает:
    answer : str
        Предложение, очищенное от ошибок
    """

    encodings = tokenizer_M100_spell(sentence_in, return_tensors="pt")

    generated_tokens = model_M100_spell.generate(**encodings, 
                                                 forced_bos_token_id=tokenizer_M100_spell.get_lang_id("ru"), 
                                                 max_new_tokens = 200)
    
    answer = tokenizer_M100_spell.batch_decode(generated_tokens, skip_special_tokens=True)
    
    answer = re.sub('[a-zA-Z]+', '', answer[0]).strip()

    return answer

In [6]:
sentence = "Основая цель мероприятия 5 орпеля 2020 - практичиская оттработка навыкоф по ока занию помощь гражданов, попавшим в ДТП, а также повышение и совершенствование уровня профессиональной подготовки сотрудников МЧС при проведении аварийно-спасательных работ по ликвидации последствий дорожно-транспортных проишествий, сокращение временной показатель реагирование."

print("Input:", sentence)
print("Output:", correct_errors(sentence_in=sentence))

Input: Основая цель мероприятия 5 орпеля 2020 - практичиская оттработка навыкоф по ока занию помощь гражданов, попавшим в ДТП, а также повышение и совершенствование уровня профессиональной подготовки сотрудников МЧС при проведении аварийно-спасательных работ по ликвидации последствий дорожно-транспортных проишествий, сокращение временной показатель реагирование.
Output: ная цель мероприятия 5 апреля 2020 - практическая отработка навыков по оказанию помощь гражданам, попавшим в ДТП, а также повышение и совершенствование уровня профессиональной подготовки сотрудников МЧС при проведении аварийно-спасательных работ по ликвидации последствий дорожно-транспортных происшествий, сокращение временной показатель реагирование.


In [7]:
sentence = "Фомилию, имя, отчество не изменял"

print("Input:", sentence)
print("Output:", correct_errors(sentence_in=sentence))

Input: Фомилию, имя, отчество не изменял
Output: Фамилию, имя, отчество не изменял


In [8]:
sentence = "Студент, Уфимский фелеал Масковского нифтеного института им. Академика И.М. Губкина"

print("Input:", sentence)
print("Output:", correct_errors(sentence_in=sentence))

Input: Студент, Уфимский фелеал Масковского нифтеного института им. Академика И.М. Губкина
Output: , Уфимский филиал Московского нефтяного института им. Академика И.М. Губкина


In [9]:
sentence = "Имею загррничный паспорт 6543217 89, ОУМС пос. Хор Хабаровского края, 04.02.2012"

print("Input:", sentence)
print("Output:", correct_errors(sentence_in=sentence))

Input: Имею загррничный паспорт 6543217 89, ОУМС пос. Хор Хабаровского края, 04.02.2012
Output: Имею загрничный паспорт 654321789, ОУМС пос. Хор Хабаровского края, 04.02.2012


In [17]:
sentence = "Калужская обл., г. Обнинск, ул. Гагарина, д. 5, кв.52"

print("Input:", sentence)
print("Output:", correct_errors(sentence_in=sentence))

Input: Калужская обл., г. Обнинск, ул. Гагарина, д. 5, кв.52
Output: Калужская обл., г. Обнинск, ул. Гагарина, д. 5, кв.52


Наблюдаются некоторые галлюционации у модели, однако в целом качество исправления опечаток приемлемо.

## NER 

### Общий раздел

Несколько общих функций, используемых при обучении NER моделей.   
Примеры и процедура для кода взяты отсюда: https://huggingface.co/learn/nlp-course/chapter7/2

In [2]:
def align_labels_with_tokens(labels, word_ids, label_names):

    new_labels = []
    current_word = None
    
    for word_id in word_ids:
        if word_id != current_word:

            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            # if label % 2 == 1:
            #     if label < len(label_names)-1:
            #         label += 1
            new_labels.append(label)

    return new_labels

In [3]:
def compute_metrics(eval_preds, label_names, metric):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [4]:
def postprocess(predictions, labels, label_names):
    
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

### NER (для имен)

#### Настройки

В качестве основной модели мы используем https://huggingface.co/ai-forever/ruBert-base. Эту модель также использовали при тонкой настройке модели viktoroo/sberbank-rubert-base-collection3, которая заточена на NER задачи. Для нашего проекта мы также будем делать тонкую настройку ruBert модели, но для конкретной задачи - вычленения из имени фамилии, имени и отчества.

Примеры кода и процедура взяты отсюда: https://huggingface.co/learn/nlp-course/chapter7/2

In [5]:
# path_to_model = "viktoroo/sberbank-rubert-base-collection3" 
# path_to_tokenizer = "viktoroo/sberbank-rubert-base-collection3"

path_to_model = "ai-forever/ruBert-base" 
path_to_tokenizer = "ai-forever/ruBert-base"

# path_to_model = "DeepPavlov/rubert-base-cased" 
# path_to_tokenizer = "DeepPavlov/rubert-base-cased"

In [6]:
tokenizer_NER = AutoTokenizer.from_pretrained(path_to_tokenizer, use_fast=True)

В нашем случае достаточно будет создавать только три сущности: имена, фамилии и отчества. Нулевую сущность мы можем опустить, так как в обрабатываемом документе в соответствующих ячейках всегда фигурируют только эти элементы.

In [17]:
label_names = ['PER-NAME', 'PER-SURN', 'PER-PATR']

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [18]:
def tokenize_and_align_labels(examples, tokenizer = tokenizer_NER):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids, label_names))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [19]:
model_NER = AutoModelForTokenClassification.from_pretrained(path_to_model,
                                                               id2label=id2label,
                                                               label2id=label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model_NER

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [21]:
tokenizer_NER

BertTokenizerFast(name_or_path='ai-forever/ruBert-base', vocab_size=120138, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

Проверка модели (закомментируем)

In [ ]:
# model_NER.save_pretrained("model/model_NER_custom/")
# tokenizer_NER.save_pretrained("model/tokenizer_NER_custom/")

In [ ]:
# для обычной модели

# sentence = "Меня зовут Вольфганг и я живу в Берлине"

# print("Input:", sentence)

# nlp = pipeline("ner", model=model_NER, tokenizer=tokenizer_NER)

# ner_results = nlp(sentence)

# print("Output:", ner_results)

In [ ]:
# "".join([elem['word'] for elem in ner_results[:2]]).replace('#', '').capitalize()

In [ ]:
# "".join([elem['word'] for elem in ner_results[2:]]).replace('#', '').capitalize()

#### Обучение

Подгружаем сгенерированные данные. Алгоритм генерации описан в блокноте "Data parser.ipynb".

In [12]:
names_dataset = pd.read_json("data/names/names_train_large.json")
print(names_dataset.shape)

(40000, 2)


Разделим набор данных на обучающую, валидационную и тестовую выборки.

In [13]:
names_dataset['mode'] = np.random.choice(['train', "val", "test"], size = names_dataset.shape[0], p = [0.9, 0.05, 0.05])

In [14]:
names_dataset['ner_tags'] = names_dataset['ner_tags'].map(lambda x: [i-1 for i in x])
names_dataset.head()

,tokens,ner_tags,mode
0,"[Олимпович, Аввакум, Копылов, Калашников]","[2, 0, 1, 1]",train
1,"[Горюнов, Арсентий, Соболев, Астафьев, Пахомович]","[1, 0, 1, 1, 2]",val
2,"[Фотиевна, Гертруда, Чижова]","[2, 0, 1]",train
3,"[Троицкий, Юзеф, Бенедиктович]","[1, 0, 2]",train
4,"[Людмила, Фадеева, Прохоровна, Грибкова]","[0, 1, 2, 1]",train


In [15]:
trdf = Dataset.from_pandas(names_dataset[names_dataset['mode'] == "train"])
vldf = Dataset.from_pandas(names_dataset[names_dataset['mode'] == "val"])
tedf = Dataset.from_pandas(names_dataset[names_dataset['mode'] == "test"])

dataset_names = DatasetDict({"train": trdf, "validation": vldf, "test": tedf})
dataset_names

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 36015
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 2027
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 1958
    })
})

In [22]:
tokenized_datasets = dataset_names.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_names["train"].column_names
)

Map:   0%|          | 0/36015 [00:00<?, ? examples/s]

Map:   0%|          | 0/2027 [00:00<?, ? examples/s]

Map:   0%|          | 0/1958 [00:00<?, ? examples/s]

In [23]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer_NER)

In [24]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(4)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    2,    2,    2,    0,    0,    0,    1,    1,    1,    1, -100],
        [-100,    2,    2,    2,    0,    0,    1,    1, -100, -100, -100, -100],
        [-100,    1,    1,    1,    0,    0,    2,    2, -100, -100, -100, -100],
        [-100,    0,    0,    1,    1,    2,    2,    2,    1,    1, -100, -100]])

In [25]:
for i in range(4):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 2, 2, 2, 0, 0, 0, 1, 1, 1, 1, -100]
[-100, 2, 2, 2, 0, 0, 1, 1, -100]
[-100, 1, 1, 1, 0, 0, 2, 2, -100]
[-100, 0, 0, 1, 1, 2, 2, 2, 1, 1, -100]


In [26]:
metric = evaluate.load("seqeval")

In [27]:
dataset_names

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 36015
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 2027
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 1958
    })
})

Проверим, правильно ли встают коды сущностей.

In [28]:
labels = dataset_names["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['PER-PATR', 'PER-NAME', 'PER-SURN', 'PER-SURN']

In [29]:
labels = dataset_names["train"][1]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['PER-PATR', 'PER-NAME', 'PER-SURN']

In [30]:
labels = dataset_names["train"][2]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['PER-SURN', 'PER-NAME', 'PER-PATR']

In [33]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PER-SURN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PER-NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PER-PATR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

{'NAME': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PATR': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'SURN': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.6666666666666666}

In [34]:
model_NER.config.num_labels

3

In [35]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=4,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    collate_fn=data_collator, 
    batch_size=8
)

In [36]:
optimizer = AdamW(model_NER.parameters(), lr=5e-05, betas=(0.9,0.999), eps=1e-08)

In [37]:
# accelerator = Accelerator(cpu=True)
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model_NER, optimizer, train_dataloader, eval_dataloader
)

In [38]:
train_dataloader.device

device(type='cuda')

In [39]:
model.device

device(type='cuda', index=0)

In [40]:
num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

cos_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=0.1,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = "model/bert-finetuned-ner-names-accelerate"

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training

    model.train()
    
    for i, batch in enumerate(train_dataloader):
        
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        cos_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
        
    model.eval()
    
    for batch in eval_dataloader:

        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered, label_names)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer_NER.save_pretrained(output_dir)

#### Тест

In [41]:
path_to_model_NER_names = "model/bert-finetuned-ner-names-accelerate" 

In [42]:
## NER для имен
label_names = ['PER-NAME', 'PER-SURN', 'PER-PATR']

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model_NER = AutoModelForTokenClassification.from_pretrained(path_to_model_NER_names,
                                                               id2label=id2label,
                                                               label2id=label2id)
tokenizer_NER = AutoTokenizer.from_pretrained(path_to_model_NER_names, use_fast=True)

model_NER.cpu()

token_classifier = pipeline(
    "token-classification", model=model_NER, aggregation_strategy="simple", tokenizer=tokenizer_NER
)

In [43]:
print(token_classifier("Макаров Андрей Николаевич"))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'SURN', 'score': 0.9999862, 'word': 'мака', 'start': 0, 'end': 4}, {'entity_group': 'PATR', 'score': 0.9999906, 'word': '##ров', 'start': 4, 'end': 7}, {'entity_group': 'NAME', 'score': 0.9999775, 'word': 'андреи', 'start': 8, 'end': 14}, {'entity_group': 'PATR', 'score': 0.99999034, 'word': 'николаевич', 'start': 15, 'end': 25}]


In [44]:
print(token_classifier("Окрошковская Елена Викторовна"))

print(token_classifier("Кобаева Виола Ролландовна"))

print(token_classifier("Макарова (Пурпурова) Валентина Михайловна"))

[{'entity_group': 'SURN', 'score': 0.9999864, 'word': 'окро', 'start': 0, 'end': 4}, {'entity_group': 'PATR', 'score': 0.9999907, 'word': '##шковская', 'start': 4, 'end': 12}, {'entity_group': 'NAME', 'score': 0.9999781, 'word': 'елена', 'start': 13, 'end': 18}, {'entity_group': 'PATR', 'score': 0.99999046, 'word': 'викторовна', 'start': 19, 'end': 29}]
[{'entity_group': 'SURN', 'score': 0.9999862, 'word': 'коба', 'start': 0, 'end': 4}, {'entity_group': 'PATR', 'score': 0.9999906, 'word': '##ева', 'start': 4, 'end': 7}, {'entity_group': 'NAME', 'score': 0.9999781, 'word': 'виола', 'start': 8, 'end': 13}, {'entity_group': 'PATR', 'score': 0.99999046, 'word': 'ролландовна', 'start': 14, 'end': 25}]
[{'entity_group': 'SURN', 'score': 0.9999865, 'word': 'мака', 'start': 0, 'end': 4}, {'entity_group': 'PATR', 'score': 0.9999907, 'word': '##рова', 'start': 4, 'end': 8}, {'entity_group': 'SURN', 'score': 0.8506638, 'word': '( пурпур', 'start': 9, 'end': 16}, {'entity_group': 'PATR', 'score': 

In [45]:
def name_reconstruct(name: str) -> str:

    """
    Функция для исправления формата имен в формат ФИО 
    В случае, если в тексте распознается более 1 фамилии, то используется формат 
        Ф (Ф1, Ф2, ... - при наличии старых фамилий) И О

    Параметры:
    name : str
        Строка с именем

    Возвращает:
    string_out : str
        Строка с именем требуемого формата
    """

    # создание словаря для сортировки элементов имени
    entities = ['SURN', 'NAME', 'PATR']
    sort_dict = {key: elem for elem, key in list(enumerate(entities))}

    name_tokens = re.findall("[а-яА-ЯЁё\-]+", name)

    NER_output = token_classifier(name_tokens)

    name_classes =  np.array([elem[0]['entity_group'] for elem in NER_output])

    # переформирование имени
    string_out = " ".join([x for _, x in sorted(zip(name_classes, name_tokens), key = lambda pair: sort_dict[pair[0]])])

    nameparts_counts = np.unique(name_classes, return_counts=True)

    # В случае, если больше одной фамилии - фамилии, следующие после 1й заключить в скобки
    if "SURN" in nameparts_counts[0] and nameparts_counts[1][-1] > 1:

        surnames = string_out.split()[:nameparts_counts[1][-1]]
        other_name_part = string_out.split()[nameparts_counts[1][-1]:]

        string_out = surnames[0] + " (" + ", ".join(surnames[1:]) + ") " + " ".join(other_name_part)

    return(string_out)

In [46]:
name_reconstruct("Пётр Петрович Семёнов-Тян-Шанский")

'Семёнов-Тян-Шанский Пётр Петрович'

In [47]:
name_reconstruct("Валентина (Пурпурова, Ольшанская)")

'Пурпурова (Ольшанская) Валентина'

Все работает корректно.

### NER (для адресов)

#### Настройки

В целом, процедура для обучения NER для распознвания адресов идентична той, что мы провели для модели для распознавания имен. Однако, мы использовали другую модель - на этот раз модель, принимающую во внимание регистр символов. Мы предполагаем, что такая модель будет гораздо лучше распознвать элементы адреса.

In [ ]:
# sentence = "Калужская обл., г. Обнинск, ул. Аксёнова, д. 33, кв.21, прибыла из г. Воскресенск Московской обл. в 1990 г."

# print("Input:", sentence)

# nlp = pipeline("ner", model=model_NER, tokenizer=tokenizer_NER)

# ner_results = nlp(sentence)

# print("Output:", ner_results)

In [ ]:
# [elem['word'].replace("#", "") for elem in ner_results if elem['entity'] in ['B-LOC', "I-LOC"]]

In [5]:
# path_to_model = "ai-forever/ruBert-large" 
# path_to_tokenizer = "ai-forever/ruBert-large"

path_to_model = "DeepPavlov/rubert-base-cased" 
path_to_tokenizer = "DeepPavlov/rubert-base-cased"

In [6]:
tokenizer_NER = AutoTokenizer.from_pretrained(path_to_tokenizer, use_fast=True)

In [7]:
label_names = ["O",
               "LOC-REG", 
               "LOC-DIST", 
               "LOC-SETL", 
               "LOC-CDIST", 
               "LOC-STRT", 
               "LOC-HOUS", 
               "LOC-FLAT"]

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [8]:
def tokenize_and_align_labels(examples, tokenizer = tokenizer_NER):

    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    
    all_labels = examples["ner_tags"]
    new_labels = []

    for i, labels in enumerate(all_labels):
        if i == 4: print(labels)
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids, label_names))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
model_NER_adr = AutoModelForTokenClassification.from_pretrained(path_to_model,
                                                               id2label=id2label,
                                                               label2id=label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Обучение

Загружаем набор данных (алгоритм генерации описан в блокноте "Data parser.ipynb")

In [10]:
adr_dataset = pd.read_json("data/addresses/addresses_train_med_extra.json")
print(adr_dataset.shape)

(1200, 2)


In [11]:
adr_dataset['mode'] = np.random.choice(['train', "val", "test"], size = adr_dataset.shape[0], p = [0.9, 0.05, 0.05])
adr_dataset['ner_tags'] = adr_dataset['ner_tags'].map(lambda x: [i for i in x])
adr_dataset.head()

,tokens,ner_tags,mode
0,"[Семиэтажное, здание, находилось, по, адресу:,...","[0, 0, 0, 0, 0, 4, 4, 1, 1, 2, 2, 3, 3, 0, 0, ...",train
1,"[Проживает, по, адресу:, Нагорское, Деревня, д...","[0, 0, 0, 4, 4, 6, 6, 7, 7, 5, 5, 2, 2, 1, 1]",train
2,"[Проживает, по, адресу:, Боханский, р-н, Тойси...","[0, 0, 0, 2, 2, 3, 3, 6, 6, 5, 5, 7, 7, 1, 1, ...",train
3,"[Выставочный, зал, находится, по, адресу, Чишм...","[0, 0, 0, 0, 0, 2, 2, 1, 1, 3, 3, 4, 4]",train
4,"[Проживает, по, адресу:, Солтонский, р-н, Стер...","[0, 0, 0, 2, 2, 3, 3, 1, 1, 4, 4, 0, 0, 0, 0, ...",train


In [12]:
trdf = Dataset.from_pandas(adr_dataset[adr_dataset['mode'] == "train"])
vldf = Dataset.from_pandas(adr_dataset[adr_dataset['mode'] == "val"])
tedf = Dataset.from_pandas(adr_dataset[adr_dataset['mode'] == "test"])

dataset_adr = DatasetDict({"train": trdf, "validation": vldf, "test": tedf})
dataset_adr

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 1072
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 64
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'mode', '__index_level_0__'],
        num_rows: 64
    })
})

In [13]:
tokenized_datasets = dataset_adr.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_adr["train"].column_names
)

Map:   0%|          | 0/1072 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[0, 0, 0, 2, 2, 3, 3, 1, 1, 4, 4, 0, 0, 0, 0, 3, 3, 7, 7, 2, 2, 6, 6, 4, 4, 5, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 4, 4, 3, 3, 7, 7, 6, 6, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Map:   0%|          | 0/64 [00:00<?, ? examples/s]

[0, 0, 0, 7, 7, 4, 4, 6, 6, 5, 5, 3, 3, 1, 1, 2, 2]


Map:   0%|          | 0/64 [00:00<?, ? examples/s]

[0, 0, 0, 0, 0, 7, 7, 6, 6, 5, 5, 3, 3, 4, 4, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0]


In [14]:
print(tokenized_datasets['train'][4]['labels'])

[-100, 0, 0, 0, 0, 2, 2, 2, 2, 2, 3, 3, 1, 1, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 7, 7, 7, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, -100]


In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer_NER)
batch = data_collator([tokenized_datasets["train"][i] for i in range(5)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    4,    4,    4,    1,
            1,    1,    1,    1,    2,    2,    2,    2,    2,    3,    3,    3,
            3,    3,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    4,    4,    4,    6,    6,    7,    7,
            7,    5,    5,    2,    2,    2,    1,    1, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    2,    2,    2,    2,    2,    3,    3,
            3,    3,    3,    3,    6,    6,    6,    5,    5,    7,    7,    7,
            1,    1,    4,    4,    4,    4,    0,    0,    0,    0,    1,    1,
            4,   

In [16]:
metric = evaluate.load("seqeval")

In [17]:
labels = dataset_adr["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'LOC-CDIST',
 'LOC-CDIST',
 'LOC-REG',
 'LOC-REG',
 'LOC-DIST',
 'LOC-DIST',
 'LOC-SETL',
 'LOC-SETL',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [18]:
predictions = labels.copy()
predictions[2] = "LOC-SETL"
metric.compute(predictions=[predictions], references=[labels])

c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: LOC-CDIST seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: LOC-REG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: LOC-DIST seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: LOC-SETL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'CDIST': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'DIST': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'REG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'SETL': {'precision': 0.5,
  'recall': 1.0,
  'f1': 0.6666666666666666,
  'number': 1},
 'overall_precision': 0.8,
 'overall_recall': 1.0,
 'overall_f1': 0.888888888888889,
 'overall_accuracy': 0.9473684210526315}

In [19]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=4,
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], 
    collate_fn=data_collator, 
    batch_size=8
)

In [20]:
optimizer = AdamW(model_NER_adr.parameters(), lr=5e-05, betas=(0.9,0.999), eps=1e-08)

In [21]:
# accelerator = Accelerator(cpu=True)
accelerator = Accelerator()

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model_NER_adr, optimizer, train_dataloader, eval_dataloader
)

In [22]:
train_dataloader.device

device(type='cuda')

In [23]:
model.device

device(type='cuda', index=0)

In [49]:
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

cos_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=0.1,
    num_training_steps=num_training_steps,
)

In [50]:
output_dir = "model/test/bert-finetuned-ner-addresses-accelerate"

In [51]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training

    # model.train()
    
    for i, batch in enumerate(train_dataloader):

        # print(i, batch['labels'])
        
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        cos_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
        
    model.eval()
    
    for batch in eval_dataloader:

        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered, label_names)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer_NER.save_pretrained(output_dir)

  0%|          | 0/2680 [00:00<?, ?it/s]

epoch 0: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 1: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 2: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 3: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 4: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 5: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 6: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.9865384615384616, 'accuracy': 0.9967453213995118}
epoch 7: {'precision': 0.9903474903474904, 'recall': 0.9827586206896551, 'f1': 0.98

#### Тест

Проверим, как хорошо работает наша модель.

In [52]:
path_to_model_NER_addresses = "model/test/bert-finetuned-ner-addresses-accelerate" 

In [53]:
## NER для адресов
label_names = ["O",
               "LOC-REG", 
               "LOC-DIST", 
               "LOC-SETL", 
               "LOC-CDIST", 
               "LOC-STRT", 
               "LOC-HOUS", 
               "LOC-FLAT"]

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model_NER_adr = AutoModelForTokenClassification.from_pretrained(path_to_model_NER_addresses,
                                                               id2label=id2label,
                                                               label2id=label2id)

tokenizer_NER_adr = AutoTokenizer.from_pretrained(path_to_model_NER_addresses, use_fast=True)


In [54]:
# path_to_base_model = "viktoroo/sberbank-rubert-base-collection3" 
# path_to_base_tokenizer = "viktoroo/sberbank-rubert-base-collection3"

In [55]:
# tokenizer_base_NER = AutoTokenizer.from_pretrained(path_to_base_tokenizer, use_fast=True)
# model_base_NER = AutoModelForTokenClassification.from_pretrained(path_to_base_model)

In [56]:
# sentence = "Калужская обл., г. Обнинск, ул. Аксёнова, д. 33, кв.21, прибыла из г. Воскресенск Московской обл. в 1990 г."

# print("Input:", sentence)

# nlp = pipeline("ner", model=model_base_NER, tokenizer=tokenizer_base_NER)

# ner_results = nlp(sentence)

# [print(i) for i in ner_results]

In [57]:
model_NER_adr.cpu()

token_classifier = pipeline(
    "token-classification", model=model_NER_adr, aggregation_strategy="simple", tokenizer=tokenizer_NER_adr
)

In [58]:
token_classifier("Калужская обл., г. Обнинск, ул. Университетская, д.50")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'REG',
  'score': 0.99929416,
  'word': 'Калужская обл.,',
  'start': 0,
  'end': 15},
 {'entity_group': 'SETL',
  'score': 0.9966365,
  'word': 'г. Обнинск',
  'start': 16,
  'end': 26},
 {'entity_group': 'STRT',
  'score': 0.8942354,
  'word': ', ул. Университетская,',
  'start': 26,
  'end': 48},
 {'entity_group': 'HOUS',
  'score': 0.9999078,
  'word': 'д. 50',
  'start': 49,
  'end': 53}]

In [59]:
[print(i) for i in token_classifier(("Калужская обл., г. Обнинск, ул. Университетская, д.50").split(", "))]

[{'entity_group': 'REG', 'score': 0.9996962, 'word': 'Калужская обл.', 'start': 0, 'end': 14}]
[{'entity_group': 'SETL', 'score': 0.9986812, 'word': 'г. Обнинск', 'start': 0, 'end': 10}]
[{'entity_group': 'STRT', 'score': 0.99980533, 'word': 'ул. Университетская', 'start': 0, 'end': 19}]
[{'entity_group': 'HOUS', 'score': 0.9998743, 'word': 'д. 50', 'start': 0, 'end': 4}]


[None, None, None, None]

In [60]:
[print(i) for i in token_classifier(("Проживала по адресу: Республика Башкортостан, г. Салават, б-р Космонавтов, д. 14, кв. 67"))]

{'entity_group': 'REG', 'score': 0.9919765, 'word': 'Республика Башкортостан,', 'start': 21, 'end': 45}
{'entity_group': 'SETL', 'score': 0.89499885, 'word': 'г. Салават,', 'start': 46, 'end': 57}
{'entity_group': 'STRT', 'score': 0.9946023, 'word': 'б - р Космонавтов,', 'start': 58, 'end': 74}
{'entity_group': 'HOUS', 'score': 0.9998271, 'word': 'д. 14,', 'start': 75, 'end': 81}
{'entity_group': 'FLAT', 'score': 0.9998968, 'word': 'кв. 67', 'start': 82, 'end': 88}


[None, None, None, None, None]

In [61]:
[print(i) for i in token_classifier(("Проживала по адресу: Республика Башкортостан, г. Салават, б-р Космонавтов, д. 14, кв. 67").split(", "))]

[{'entity_group': 'REG', 'score': 0.99983895, 'word': 'Республика Башкортостан', 'start': 21, 'end': 44}]
[{'entity_group': 'SETL', 'score': 0.98052216, 'word': 'г. Салават', 'start': 0, 'end': 10}]
[{'entity_group': 'STRT', 'score': 0.9761292, 'word': 'б - р Космонавтов', 'start': 0, 'end': 15}]
[{'entity_group': 'HOUS', 'score': 0.99987745, 'word': 'д. 14', 'start': 0, 'end': 5}]
[{'entity_group': 'FLAT', 'score': 0.9998687, 'word': 'кв. 67', 'start': 0, 'end': 6}]


[None, None, None, None, None]

In [62]:
[print(i) for i in token_classifier(("Башкирской АССР, г. Уфа, ул. Космонавтов, д. 1").split(", "))]

[{'entity_group': 'REG', 'score': 0.9996654, 'word': 'Башкирской АССР', 'start': 0, 'end': 15}]
[{'entity_group': 'SETL', 'score': 0.81834745, 'word': 'г.', 'start': 0, 'end': 2}, {'entity_group': 'REG', 'score': 0.48711446, 'word': 'Уфа', 'start': 3, 'end': 6}]
[{'entity_group': 'STRT', 'score': 0.99980766, 'word': 'ул. Космонавтов', 'start': 0, 'end': 15}]
[{'entity_group': 'HOUS', 'score': 0.9997428, 'word': 'д. 1', 'start': 0, 'end': 4}]


[None, None, None, None]

In [63]:
[print(i) for i in \
 token_classifier(("Калужская обл., г. Обнинск, ул. Аксёнова, д. 33, кв.21, прибыла из г. Воскресенск Московской обл. в 1990 г").split(", "))]

[{'entity_group': 'REG', 'score': 0.9996962, 'word': 'Калужская обл.', 'start': 0, 'end': 14}]
[{'entity_group': 'SETL', 'score': 0.9986812, 'word': 'г. Обнинск', 'start': 0, 'end': 10}]
[{'entity_group': 'STRT', 'score': 0.9997964, 'word': 'ул. Аксёнова', 'start': 0, 'end': 12}]
[{'entity_group': 'HOUS', 'score': 0.99987406, 'word': 'д. 33', 'start': 0, 'end': 5}]
[{'entity_group': 'FLAT', 'score': 0.99983984, 'word': 'кв. 21', 'start': 0, 'end': 5}]
[{'entity_group': 'SETL', 'score': 0.9339662, 'word': 'г. Воскресенск', 'start': 11, 'end': 25}, {'entity_group': 'REG', 'score': 0.9108238, 'word': 'Московской обл.', 'start': 26, 'end': 41}]


[None, None, None, None, None, None]

In [64]:
[print(i) for i in \
 token_classifier(("г. Москва, ул. Новочерёмушкинская, д. 27, кв. 154, прибыл из г. Обнинска Калужской обл. в 2020 г.").split(", "))]

[{'entity_group': 'SETL', 'score': 0.7186454, 'word': 'г. Москва', 'start': 0, 'end': 9}]
[{'entity_group': 'STRT', 'score': 0.9993895, 'word': 'ул. Новочерёмушкинская', 'start': 0, 'end': 22}]
[{'entity_group': 'HOUS', 'score': 0.99986935, 'word': 'д. 27', 'start': 0, 'end': 5}]
[{'entity_group': 'FLAT', 'score': 0.99985933, 'word': 'кв. 154', 'start': 0, 'end': 7}]
[{'entity_group': 'SETL', 'score': 0.972817, 'word': 'г. Обнинска', 'start': 10, 'end': 21}, {'entity_group': 'REG', 'score': 0.99963623, 'word': 'Калужской обл.', 'start': 22, 'end': 36}]


[None, None, None, None, None]

In [65]:
[print(i) for i in token_classifier(("г. Москва, ул. Новочерёмушкинская, д. 27, кв. 154, прибыл из г. Обнинска Калужской обл. в 2020 г."))]

{'entity_group': 'SETL', 'score': 0.5084862, 'word': 'г', 'start': 0, 'end': 1}
{'entity_group': 'REG', 'score': 0.7918929, 'word': '. Москва', 'start': 1, 'end': 9}
{'entity_group': 'STRT', 'score': 0.9964458, 'word': 'ул. Новочерёмушкинская,', 'start': 11, 'end': 34}
{'entity_group': 'HOUS', 'score': 0.9641217, 'word': 'д. 27,', 'start': 35, 'end': 41}
{'entity_group': 'FLAT', 'score': 0.9998932, 'word': 'кв. 154', 'start': 42, 'end': 49}
{'entity_group': 'SETL', 'score': 0.99644387, 'word': 'г. Обнинска', 'start': 61, 'end': 72}
{'entity_group': 'REG', 'score': 0.99984425, 'word': 'Калужской обл.', 'start': 73, 'end': 87}


[None, None, None, None, None, None, None]

In [66]:
def address_reconstruct(address: str) -> str:

    """
    Функция для исправления формата адреса в формат Регион, район, город/поселок, улица, дом, квартира 

    Параметры:
    address : str
        Строка, содержащая адрес

    Возвращает:
    string_out : str
        Строка с адресом требуемого формата
    """

    # создание словаря для сортировки элементов адреса
    entities = ["O", "REG", "DIST", "SETL", "CDIST", "STRT", "HOUS", "FLAT"]
    
    sort_dict = {key: elem for elem, key in list(enumerate(entities))}

    adr_tokens = address.strip().split(", ")

    # print(adr_tokens)

    NER_output = list(token_classifier(adr_tokens))

    addresses = [[]]
    adr_entities = [[]]
    i = 0

    for token, elem in zip(adr_tokens, NER_output):

        if len(elem) > 1:

            if elem[0]["start"] > 0:

                i += 1
                addresses.append([token[:elem[0]["start"]]])
                adr_entities.append(["O"])

            for subelem in elem:

                addresses[i].append(subelem["word"])
                adr_entities[i].append(subelem["entity_group"])

            if elem[-1]["end"] < len(token)-1:

                addresses[i].append(token[elem[-1]["end"]:])
                adr_entities[i].append("O")

        else:
            addresses[i].append(token)
            adr_entities[i].append(elem[0]["entity_group"])

    print(addresses)
    print(adr_entities)

    final_list = []

    for adr_lst, entity in zip(addresses, adr_entities):      

        print("tokens:", adr_lst)
        print("entities:", entity)    

        if entity[0] == "O":
                idx_start = 1
                left = adr_lst[0]
        else:
                idx_start = 0
                left = ""

        if entity[-1] == "O":
                idx_end = len(entity) - 1
                right = adr_lst[-1]
        else:
                idx_end = len(entity)
                right = ""

        adr_sorted = [x for _, x in sorted(zip(entity[idx_start:idx_end], adr_lst[idx_start:idx_end]), \
                                            key = lambda pair: sort_dict[pair[0]])]

        final_list.append(left + ", ".join(adr_sorted) + right)
        
        # print(final_list)

    # # переформирование адреса
    string_out = ", ".join(final_list)

    return(string_out)

In [67]:
adr_text = "ул. Новочерёмушкинская, д. 27, кв. 154, г. Москва, прибыл из г. Обнинска Калужской обл. в 2020 г."
address_reconstruct(adr_text)

[['ул. Новочерёмушкинская', 'д. 27', 'кв. 154', 'г. Москва'], ['прибыл из ', 'г. Обнинска', 'Калужской обл.', ' в 2020 г.']]
[['STRT', 'HOUS', 'FLAT', 'SETL'], ['O', 'SETL', 'REG', 'O']]
tokens: ['ул. Новочерёмушкинская', 'д. 27', 'кв. 154', 'г. Москва']
entities: ['STRT', 'HOUS', 'FLAT', 'SETL']
tokens: ['прибыл из ', 'г. Обнинска', 'Калужской обл.', ' в 2020 г.']
entities: ['O', 'SETL', 'REG', 'O']


'г. Москва, ул. Новочерёмушкинская, д. 27, кв. 154, прибыл из Калужской обл., г. Обнинска в 2020 г.'

In [68]:
adr_text = "г. Москва, Старокалужское ш., д. 62, корп. 4, стр. 1"
address_reconstruct(adr_text)

[['г. Москва', 'Старокалужское ш.', 'д. 62', 'корп. 4', 'стр. 1']]
[['SETL', 'SETL', 'HOUS', 'HOUS', 'HOUS']]
tokens: ['г. Москва', 'Старокалужское ш.', 'д. 62', 'корп. 4', 'стр. 1']
entities: ['SETL', 'SETL', 'HOUS', 'HOUS', 'HOUS']


'г. Москва, Старокалужское ш., д. 62, корп. 4, стр. 1'

In [69]:
adr_text = "г. Москва, ул. Ярцевская, д. 29, корп.2, кв. 147"
address_reconstruct(adr_text)

[['г. Москва', 'ул. Ярцевская', 'д. 29', 'корп.2', 'кв. 147']]
[['SETL', 'STRT', 'HOUS', 'HOUS', 'FLAT']]
tokens: ['г. Москва', 'ул. Ярцевская', 'д. 29', 'корп.2', 'кв. 147']
entities: ['SETL', 'STRT', 'HOUS', 'HOUS', 'FLAT']


'г. Москва, ул. Ярцевская, д. 29, корп.2, кв. 147'

In [70]:
address_reconstruct("ул. Университетская, д.50, г. Обнинск, Калужская обл.")

[['ул. Университетская', 'д.50', 'г. Обнинск', 'Калужская обл.']]
[['STRT', 'HOUS', 'SETL', 'REG']]
tokens: ['ул. Университетская', 'д.50', 'г. Обнинск', 'Калужская обл.']
entities: ['STRT', 'HOUS', 'SETL', 'REG']


'Калужская обл., г. Обнинск, ул. Университетская, д.50'

In [71]:
address_reconstruct("ул. Ярцевская, д. 29, корп.2, кв. 147, г. Москва")

[['ул. Ярцевская', 'д. 29', 'корп.2', 'кв. 147', 'г. Москва']]
[['STRT', 'HOUS', 'HOUS', 'FLAT', 'SETL']]
tokens: ['ул. Ярцевская', 'д. 29', 'корп.2', 'кв. 147', 'г. Москва']
entities: ['STRT', 'HOUS', 'HOUS', 'FLAT', 'SETL']


'г. Москва, ул. Ярцевская, д. 29, корп.2, кв. 147'

In [72]:
address_reconstruct("г. Хабаровск, Хабаровский край, Центральный р-н, ул. Гоголя, д. 42")

[['г. Хабаровск', 'Хабаровский край', 'Центральный р-н', 'ул. Гоголя', 'д. 42']]
[['SETL', 'REG', 'DIST', 'STRT', 'HOUS']]
tokens: ['г. Хабаровск', 'Хабаровский край', 'Центральный р-н', 'ул. Гоголя', 'д. 42']
entities: ['SETL', 'REG', 'DIST', 'STRT', 'HOUS']


'Хабаровский край, Центральный р-н, г. Хабаровск, ул. Гоголя, д. 42'

В целом, качество распознавания элементов адреса очень неплохое, однако местами встречаются ошибки, вроде распознавание районов города как районов региона. В целом такие ошибки проблематично выловить по двум причинам - в наборе данных элементы адреса специально перемешаны, поскольку целью использования модели является восстановление порядка адреса - сначала регион, потом район, населенный пункт, район населенного пункта, улица, дом, квартира. Пользователь при вводе данных в форму может менять порядок некоторых элементов.

Решение - не использовать нейросеть, или же после получения вывода нейросети провести дополнительную обработку результата, однако подобное решение требует создания валидатора адреса, что в принципе убирает необходимость в нейросети.